# Team 1 tamrin 4
- Maryam tari
- Parvin akbari
- Amirhosein abdolahi

# Importing Libraries

In [44]:
import numpy as np
import pandas as pd

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

print('Libraries loaded.')

Libraries loaded.


# Example 1: Simple use of algorithm

## Creating Dataset

In [45]:
basket_lists = [['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
                ['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
                ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
                ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
                ['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs']]

trans_encoder = TransactionEncoder()

encoded_baskets = trans_encoder.fit(basket_lists).transform(basket_lists)

dataset_df = pd.DataFrame(encoded_baskets, columns= trans_encoder.columns_)

dataset_df

,Apple,Corn,Dill,Eggs,Ice cream,Kidney Beans,Milk,Nutmeg,Onion,Unicorn,Yogurt
0,False,False,False,True,False,True,True,True,True,False,True
1,False,False,True,True,False,True,False,True,True,False,True
2,True,False,False,True,False,True,True,False,False,False,False
3,False,True,False,False,False,True,True,False,False,True,True
4,False,True,False,True,True,True,False,False,True,False,False


## Computing support values

In [46]:
apriori(dataset_df, min_support= 0.6, use_colnames= True)

,support,itemsets
0,0.8,(Eggs)
1,1.0,(Kidney Beans)
2,0.6,(Milk)
3,0.6,(Onion)
4,0.6,(Yogurt)
5,0.8,"(Eggs, Kidney Beans)"
6,0.6,"(Onion, Eggs)"
7,0.6,"(Milk, Kidney Beans)"
8,0.6,"(Onion, Kidney Beans)"
9,0.6,"(Yogurt, Kidney Beans)"


# Example2: Association rule mining

## Loading dataset

In [47]:
basket_df = pd.read_excel('Data/Online Retail.xlsx', engine='openpyxl')

In [48]:
basket_df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


### Dataset description

## Some basic exploratory

In [49]:
basket_df.shape

(541909, 8)

In [50]:
basket_df.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

In [51]:
basket_df.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [52]:
canceled = basket_df.loc[basket_df['InvoiceNo'].str.contains('C') == True]

canceled.shape

(9288, 8)

## Cleansing dataset

In [53]:
basket_df['Description'] = basket_df['Description'].str.strip()

#removing canceled records
basket_df.drop(canceled.index, inplace= True)

print(f'{canceled.shape[0]} has removed from dataset, new dataset size is {basket_df.shape[0]}')

9288 has removed from dataset, new dataset size is 532621


## Preparing dataset for analysing

### Selecting one region (Germany)

In [54]:
france_basket = basket_df.loc[basket_df['Country'] == 'France']

france_basket.shape

(8408, 8)

### Creating transaction, items set

In [64]:
basket = []
transaction_nom = []

for transaction, items in france_basket.groupby('InvoiceNo').groups.items():
    
    products = []
    
    transaction_nom.append(transaction)
    
    for item in items:
        
        products.append(str(france_basket.loc[item]['Description']))
    
    ###   
    basket.append(products)

### creating binary dataset

In [65]:
trans_encoder = TransactionEncoder()

encoded_baskets = trans_encoder.fit(basket).transform(basket)

binary_df = pd.DataFrame(encoded_baskets, columns= trans_encoder.columns_, index= transaction_nom)

binary_df

,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,...,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,YELLOW SHARK HELICOPTER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
536370,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
536852,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
536974,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
537065,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
537463,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580986,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
581001,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
581171,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
581279,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### Association extraction(Recommendation based on antecedents)

In [66]:
frequencies_items = apriori(binary_df, min_support= 0.05, use_colnames= True)
  
rules = association_rules(frequencies_items, metric= "lift", min_threshold= 1)

rules = rules.sort_values(['confidence', 'lift'], ascending= [False, False])

rules.reset_index(inplace= True, drop= True)

rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(JUMBO BAG WOODLAND ANIMALS),(POSTAGE),0.076531,0.765306,0.076531,1.000,1.306667,0.017961,inf
1,"(PLASTERS IN TIN CIRCUS PARADE, RED TOADSTOOL ...",(POSTAGE),0.051020,0.765306,0.051020,1.000,1.306667,0.011974,inf
2,"(PLASTERS IN TIN WOODLAND ANIMALS, RED TOADSTO...",(POSTAGE),0.053571,0.765306,0.053571,1.000,1.306667,0.012573,inf
3,"(SET/20 RED RETROSPOT PAPER NAPKINS, SET/6 RED...",(SET/6 RED SPOTTY PAPER PLATES),0.102041,0.127551,0.099490,0.975,7.644000,0.086474,34.897959
4,"(SET/20 RED RETROSPOT PAPER NAPKINS, SET/6 RED...",(SET/6 RED SPOTTY PAPER CUPS),0.102041,0.137755,0.099490,0.975,7.077778,0.085433,34.489796


### Filtering results

In [67]:
condition0 = rules.antecedents.apply(lambda x: False if 'POST' in x else True)

condition1 = rules.consequents.apply(lambda x: False if 'POST' in x else True)

condition2 = rules['antecedents'].apply(lambda x: len(x) >= 2)

rules.loc[condition0 & condition1 & condition2]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1,"(PLASTERS IN TIN CIRCUS PARADE, RED TOADSTOOL ...",(POSTAGE),0.051020,0.765306,0.051020,1.000000,1.306667,0.011974,inf
2,"(PLASTERS IN TIN WOODLAND ANIMALS, RED TOADSTO...",(POSTAGE),0.053571,0.765306,0.053571,1.000000,1.306667,0.012573,inf
3,"(SET/20 RED RETROSPOT PAPER NAPKINS, SET/6 RED...",(SET/6 RED SPOTTY PAPER PLATES),0.102041,0.127551,0.099490,0.975000,7.644000,0.086474,34.897959
4,"(SET/20 RED RETROSPOT PAPER NAPKINS, SET/6 RED...",(SET/6 RED SPOTTY PAPER CUPS),0.102041,0.137755,0.099490,0.975000,7.077778,0.085433,34.489796
5,"(SET/20 RED RETROSPOT PAPER NAPKINS, SET/6 RED...",(SET/6 RED SPOTTY PAPER PLATES),0.084184,0.127551,0.081633,0.969697,7.602424,0.070895,28.790816
...,...,...,...,...,...,...,...,...,...
239,"(POSTAGE, PLASTERS IN TIN WOODLAND ANIMALS)",(RED TOADSTOOL LED NIGHT LIGHT),0.137755,0.181122,0.053571,0.388889,2.147105,0.028621,1.339981
245,"(POSTAGE, PLASTERS IN TIN CIRCUS PARADE)","(PLASTERS IN TIN WOODLAND ANIMALS, PLASTERS IN...",0.147959,0.104592,0.053571,0.362069,3.461733,0.038096,1.403613
251,"(POSTAGE, PLASTERS IN TIN CIRCUS PARADE)",(RED TOADSTOOL LED NIGHT LIGHT),0.147959,0.181122,0.051020,0.344828,1.903837,0.024222,1.249866
253,"(POSTAGE, RED TOADSTOOL LED NIGHT LIGHT)",(PLASTERS IN TIN WOODLAND ANIMALS),0.158163,0.170918,0.053571,0.338710,1.981704,0.026538,1.253733


# Excersice1

در قواعد بدست آمده از کد کالا استفاده شده است. قواعد را با استفاده از توضیح هر کد کالا برای کاربرقابل فهم کنید. به عبارت دیگر بجای کد کالا از توضیح یا نام کالا استفاده کنید. 

replacing 'Description' insted of 'StockCode'

# Excersize2

بررسی کنید قواعد بدست آمده از تحلیل سبد خرید فرانسوی های چقدر مشابه قوانین سبد خرید سوئیس هاست. 
به عبارت دیگر با چه احتمالی می توان قواعد بدست آمده از تحلیل سبد خرید فرانسویان را به سبد خرید سوئیسی ها تعمیم داد.